In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('loan_data.csv')
df.head()

,person_age,person_gender,person_education,person_income,person_emp_exp,person_home_ownership,loan_amnt,loan_intent,loan_int_rate,loan_percent_income,cb_person_cred_hist_length,credit_score,previous_loan_defaults_on_file,loan_status
0,22.0,female,Master,71948.0,0,RENT,35000.0,PERSONAL,16.02,0.49,3.0,561,No,1
1,21.0,female,High School,12282.0,0,OWN,1000.0,EDUCATION,11.14,0.08,2.0,504,Yes,0
2,25.0,female,High School,12438.0,3,MORTGAGE,5500.0,MEDICAL,12.87,0.44,3.0,635,No,1
3,23.0,female,Bachelor,79753.0,0,RENT,35000.0,MEDICAL,15.23,0.44,2.0,675,No,1
4,24.0,male,Master,66135.0,1,RENT,35000.0,MEDICAL,14.27,0.53,4.0,586,No,1


In [3]:
categorical_cols= ['person_gender', 'person_education', 'person_home_ownership', 'loan_intent', 'previous_loan_defaults_on_file']
numerical_cols = ['person_age', 'person_income', 'person_emp_exp', 'loan_amnt', 'loan_int_rate', 'loan_percent_income', 'cb_person_cred_hist_length', 'credit_score']

In [4]:
skewed_cols = ['person_age', 'person_income', 'person_emp_exp',
               'loan_amnt', 'loan_percent_income',
               'cb_person_cred_hist_length', 'credit_score']

norm_cols= ['loan_int_rate']

In [5]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PowerTransformer

In [6]:
def apply_yeo_johnson_on_left_skewed(df: pd.DataFrame, skewed_cols: list):
    pt = PowerTransformer(method='yeo-johnson')
    df_transformed = df.copy()

    for col in skewed_cols:  # Consider it left-skewed
        reshaped = df[[col]].dropna()  # Drop NaNs temporarily
        transformed = pt.fit_transform(reshaped)
        df_transformed.loc[reshaped.index, col] = transformed

    return df_transformed

In [7]:
df_transformed = apply_yeo_johnson_on_left_skewed(df, skewed_cols)

C:\Users\Akshay\AppData\Local\Temp\ipykernel_10980\2535163183.py:8: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[-1.45090001 -1.45090001 -0.08360331 ...  0.65070681  0.14896785
 -0.78360458]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_transformed.loc[reshaped.index, col] = transformed
C:\Users\Akshay\AppData\Local\Temp\ipykernel_10980\2535163183.py:8: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[-1.37672516 -2.11601292 -0.06033249 ...  0.67879598 -0.66584082
 -0.20431105]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_transformed.loc[reshaped.index, col] = transformed


In [8]:
def create_dummies(df: pd.DataFrame, drop_first: bool = True):
    df_encoded = pd.get_dummies(df, columns=categorical_cols, drop_first=drop_first, dtype=int)
    return df_encoded

In [9]:
df_final = create_dummies(df_transformed)

In [10]:
X = df_final.drop('loan_status', axis=1)
y = df_final['loan_status']

In [11]:
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE

# Step 1: Split your data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Step 2: Apply SMOTE on the training set only
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

C:\Users\Akshay\anaconda3\envs\Loan-Status-Approval\lib\site-packages\joblib\externals\loky\backend\context.py:136: UserWarning: Could not find the number of physical cores for the following reason:
[WinError 2] The system cannot find the file specified
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "C:\Users\Akshay\anaconda3\envs\Loan-Status-Approval\lib\site-packages\joblib\externals\loky\backend\context.py", line 257, in _count_physical_cores
    cpu_info = subprocess.run(
  File "C:\Users\Akshay\anaconda3\envs\Loan-Status-Approval\lib\subprocess.py", line 503, in run
    with Popen(*popenargs, **kwargs) as process:
  File "C:\Users\Akshay\anaconda3\envs\Loan-Status-Approval\lib\subprocess.py", line 971, in __init__
    self._execute_child(args, executable, preexec_fn, close_fds,
  File "C:\Users\Akshay\anaconda3\envs\Loan-Status-Approval\lib\subprocess.py", lin

In [13]:
import os
import dagshub
import mlflow
from mlflow.exceptions import MlflowException

# Configuration - Replace with your actual details
DAGSHUB_USERNAME = "Akshat-Sharma-110011"
DAGSHUB_TOKEN = "268f8944c99d48868fa3235eb38ea909e929c70c"  # Keep this secret!
REPO_NAME = "Loan-Status-Approval"

# 1. Set environment variables
os.environ["MLFLOW_TRACKING_USERNAME"] = DAGSHUB_USERNAME
os.environ["MLFLOW_TRACKING_PASSWORD"] = DAGSHUB_TOKEN

# 2. Initialize DagsHub - do this first
dagshub.init(
    repo_owner=DAGSHUB_USERNAME,
    repo_name=REPO_NAME,
    mlflow=True
)

# 3. Set the tracking URI with the correct format
mlflow.set_tracking_uri(f"https://dagshub.com/{DAGSHUB_USERNAME}/{REPO_NAME}.mlflow")

# 4. Handle experiment creation more robustly
EXPERIMENT_NAME = "CatBoost Optimizer"

try:
    # Check if experiment exists
    if not mlflow.get_experiment_by_name(EXPERIMENT_NAME):
        print(f"Experiment '{EXPERIMENT_NAME}' does not exist. Creating...")
        mlflow.create_experiment(EXPERIMENT_NAME)
    mlflow.set_experiment(EXPERIMENT_NAME)
    print(f"Successfully set experiment: {EXPERIMENT_NAME}")

except MlflowException as e:
    print(f"MLflow Error: {e}")
except Exception as e:
    print(f"Unexpected Error: {e}")

Initialized MLflow to track repo "Akshat-Sharma-110011/Loan-Status-Approval"

Repository Akshat-Sharma-110011/Loan-Status-Approval initialized!

Experiment 'CatBoost Optimizer' does not exist. Creating...
Successfully set experiment: CatBoost Optimizer


In [14]:
import logging
import time
import mlflow
import optuna
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

# Define Optuna objective function
def objective(trial):
    # Start a new child run for each Optuna trial
    with mlflow.start_run(nested=True):
        # Suggest hyperparameters
        params = {
            "iterations": trial.suggest_int("iterations", 100, 1000, step=100),
            "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
            "depth": trial.suggest_int("depth", 3, 10),
            "random_state": 42,
            "verbose": 0
        }

        # Log trial parameters
        mlflow.log_params(params)

        # Train model
        model = CatBoostClassifier(**params)
        model.fit(X_train_resampled, y_train_resampled)

        # Predict and evaluate
        y_pred = model.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)

        # Log metrics
        mlflow.log_metrics({
            "accuracy": accuracy,
            "precision": precision,
            "recall": recall,
            "f1_score": f1
        })

        # Return F1 for optimization
        return f1

# Start parent run
logging.info("Starting MLflow parent run and Optuna study...")
with mlflow.start_run(run_name="CatBoost_Optuna_HPO") as parent_run:
    start_time = time.time()

    # Log general experiment settings
    mlflow.log_param("random_state", 42)
    mlflow.log_param("test_size", 0.2)
    mlflow.set_tag("mlflow.runType", "parent")

    # Run Optuna Study
    study = optuna.create_study(direction="maximize", study_name="CatBoost Optimization")
    study.optimize(objective, n_trials=30)

    # Log best trial
    best_params = study.best_trial.params
    mlflow.log_params({f"best_{k}": v for k, v in best_params.items()})
    mlflow.log_metric("best_f1_score", study.best_value)

    # Train final model on best parameters
    logging.info("Training final model with best parameters...")
    final_model = CatBoostClassifier(**best_params, random_state=42, verbose=0)
    final_model.fit(X_train_resampled, y_train_resampled)

    y_final_pred = final_model.predict(X_test)
    final_accuracy = accuracy_score(y_test, y_final_pred)
    final_precision = precision_score(y_test, y_final_pred)
    final_recall = recall_score(y_test, y_final_pred)
    final_f1 = f1_score(y_test, y_final_pred)

    mlflow.log_metrics({
        "final_accuracy": final_accuracy,
        "final_precision": final_precision,
        "final_recall": final_recall,
        "final_f1_score": final_f1
    })

    # Log final model
    mlflow.sklearn.log_model(final_model, "final_model")

    end_time = time.time()
    logging.info(f"Total tuning + final training time: {end_time - start_time:.2f} seconds.")


C:\Users\Akshay\anaconda3\envs\Loan-Status-Approval\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-05-03 14:35:45,263 - INFO - Starting MLflow parent run and Optuna study...
[I 2025-05-03 14:35:46,997] A new study created in memory with name: CatBoost Optimization


🏃 View run masked-shoat-180 at: https://dagshub.com/Akshat-Sharma-110011/Loan-Status-Approval.mlflow/#/experiments/1/runs/3284e4c2b9ea404fa71083330868bc5e
🧪 View experiment at: https://dagshub.com/Akshat-Sharma-110011/Loan-Status-Approval.mlflow/#/experiments/1


[I 2025-05-03 14:36:05,128] Trial 0 finished with value: 0.8484693877551021 and parameters: {'iterations': 700, 'learning_rate': 0.22272277530025023, 'depth': 9}. Best is trial 0 with value: 0.8484693877551021.


🏃 View run unruly-horse-762 at: https://dagshub.com/Akshat-Sharma-110011/Loan-Status-Approval.mlflow/#/experiments/1/runs/5e34c6bc5a2a4619aaeb7e5bff95ae05
🧪 View experiment at: https://dagshub.com/Akshat-Sharma-110011/Loan-Status-Approval.mlflow/#/experiments/1


[I 2025-05-03 14:36:16,172] Trial 1 finished with value: 0.8096885813148789 and parameters: {'iterations': 500, 'learning_rate': 0.014196924848219071, 'depth': 6}. Best is trial 0 with value: 0.8484693877551021.


...

🏃 View run trusting-stork-133 at: https://dagshub.com/Akshat-Sharma-110011/Loan-Status-Approval.mlflow/#/experiments/1/runs/b4a24bb61c4c45fba1f017c57a98b9ea
🧪 View experiment at: https://dagshub.com/Akshat-Sharma-110011/Loan-Status-Approval.mlflow/#/experiments/1


[I 2025-05-03 14:41:15,026] Trial 28 finished with value: 0.816089168887812 and parameters: {'iterations': 800, 'learning_rate': 0.014792738876250879, 'depth': 6}. Best is trial 22 with value: 0.85692268173645.


🏃 View run popular-roo-530 at: https://dagshub.com/Akshat-Sharma-110011/Loan-Status-Approval.mlflow/#/experiments/1/runs/b116257b7c9c4af08255c7fbba0de317
🧪 View experiment at: https://dagshub.com/Akshat-Sharma-110011/Loan-Status-Approval.mlflow/#/experiments/1


[I 2025-05-03 14:41:20,676] Trial 29 finished with value: 0.8545595453371222 and parameters: {'iterations': 600, 'learning_rate': 0.22136631391989617, 'depth': 4}. Best is trial 22 with value: 0.85692268173645.
2025-05-03 14:41:21,414 - INFO - Training final model with best parameters...
2025/05/03 14:41:34 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025-05-03 14:41:39,095 - INFO - Total tuning + final training time: 353.25 seconds.


🏃 View run CatBoost_Optuna_HPO at: https://dagshub.com/Akshat-Sharma-110011/Loan-Status-Approval.mlflow/#/experiments/1/runs/b75db6ed31944681a9e7bfe450db8d4e
🧪 View experiment at: https://dagshub.com/Akshat-Sharma-110011/Loan-Status-Approval.mlflow/#/experiments/1
